In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!sudo update-alternatives --config python3
!sudo apt install python3-pip # Use python version 3.8
!pip install transformers==3.3.1
!pip install seqeval==1.2.2
!pip install torch

%cd gdrive/MyDrive/neural-NER/code/RoSTER/

In [ ]:
# Run on Biocaster Dataset ()
sh = """
CORPUS=bio_s_500
SEED=30
BACKBONE_DIR=out_biocaster_weak_2775_longer
TEMP_DIR=tmp_${CORPUS}_$SEED
OUT_DIR=out_$CORPUS
mkdir -p $TEMP_DIR
mkdir -p $OUT_DIR

python -u src/train.py --data_dir new_data/biocaster/strong/$CORPUS --output_dir $OUT_DIR --temp_dir $TEMP_DIR \
    --pretrained_model roberta-base --tag_scheme 'io' --max_seq_length 150 \
    --train_batch_size 32 --gradient_accumulation_steps 2 --eval_batch_size 64 \
    --noise_train_lr 3e-5 --ensemble_train_lr 1e-5 --self_train_lr 5e-7 \
    --noise_train_epochs 30 --ensemble_train_epochs 0 --self_train_epochs 0 \
    --noise_train_update_interval 60 --self_train_update_interval 100 \
    --dropout 0.1 --warmup_proportion 0.1 --seed $SEED \
    --q 0.0 --tau 0.0 --num_models 1 \
    --do_train --do_eval --eval_on "test" | tee $OUT_DIR/train_log.txt
"""
with open('script.sh', 'w') as file:
  file.write(sh)
!bash script.sh

In [ ]:
# Before bed
# Run on Wikigold Transfer
# No ensemble
#     --num_models 1 --backbone_dir_s1 out_wiki_w_ --do_strong_tuning --do_strong_tuning_eval --eval_on "test" | tee $OUT_DIR/train_log.txt
# Ensemble
#     --num_models 5 --backbone_dir out_onto_transfer_saved --do_train --do_eval --eval_on "test" | tee $OUT_DIR/train_log.txt
sh = """
CORPUS=wiki_w_bio_w_2000
SEED=30
TEMP_DIR=tmp_${CORPUS}_$SEED
OUT_DIR=out_${CORPUS}
mkdir -p $TEMP_DIR
mkdir -p $OUT_DIR

python -u src/train.py --data_dir new_data/wikigold_plus/$CORPUS --output_dir $OUT_DIR --temp_dir $TEMP_DIR \
    --pretrained_model roberta-base --tag_scheme 'io' --max_seq_length 150 \
    --train_batch_size 32 --gradient_accumulation_steps 2 --eval_batch_size 64 \
    --noise_train_lr 3e-5 --ensemble_train_lr 1e-5 --self_train_lr 5e-7 \
    --noise_train_epochs 10 --ensemble_train_epochs 10 --self_train_epochs 5 \
    --noise_train_update_interval 60 --self_train_update_interval 100 \
    --dropout 0.1 --warmup_proportion 0.1 --seed $SEED \
    --num_models 5 --tau 0.7 --q 0.7 --backbone_dir out_wiki_w --do_train --do_eval --eval_on "test" --save_model | tee $OUT_DIR/train_log.txt
"""
with open('script.sh', 'w') as file:
  file.write(sh)
!bash script.sh

In [ ]:
# Run on Wikigold Dataset
sh = """
CORPUS=wiki_w
SEED=30
TEMP_DIR=tmp_${CORPUS}_$SEED
OUT_DIR=out_$CORPUS
mkdir -p $TEMP_DIR
mkdir -p $OUT_DIR

python -u src/train.py --data_dir new_data/wikigold/weak --output_dir $OUT_DIR --temp_dir $TEMP_DIR \
    --pretrained_model roberta-base --tag_scheme 'io' --max_seq_length 150 \
    --train_batch_size 32 --gradient_accumulation_steps 1 --eval_batch_size 64 \
    --noise_train_lr 3e-5 --ensemble_train_lr 1e-5 --self_train_lr 5e-7 \
    --noise_train_epochs 5 --ensemble_train_epochs 10 --self_train_epochs 5 \
    --noise_train_update_interval 60 --self_train_update_interval 100 \
    --dropout 0.1 --warmup_proportion=0.1 --seed $SEED \
    --q 0.7 --tau 0.7 --num_models 5 \
    --do_train --do_eval --eval_on "test" --save_model | tee $OUT_DIR/train_log.txt
"""
with open('script.sh', 'w') as file:
  file.write(sh)
!bash script.sh

In [ ]:
# Run on Onto Dataset
sh = """
CORPUS=onto_w
SEED=30
TEMP_DIR=tmp_${CORPUS}_$SEED
OUT_DIR=out_$CORPUS
mkdir -p $TEMP_DIR
mkdir -p $OUT_DIR

python -u src/train.py --data_dir new_data/onto/weak --output_dir $OUT_DIR --temp_dir $TEMP_DIR \
    --pretrained_model roberta-base --tag_scheme 'io' --max_seq_length 150 \
    --train_batch_size 32 --gradient_accumulation_steps 1 --eval_batch_size 64 \
    --noise_train_lr 3e-5 --ensemble_train_lr 1e-5 --self_train_lr 5e-7 \
    --noise_train_epochs 3 --ensemble_train_epochs 2 --self_train_epochs 1 \
    --noise_train_update_interval 200 --self_train_update_interval 100 \
    --dropout 0.1 --warmup_proportion 0.1 --seed $SEED \
    --q 0.7 --tau 0.7 --num_models 3 \
    --do_train --do_eval --eval_on "test" --save_model | tee $OUT_DIR/train_log.txt
"""
with open('script.sh', 'w') as file:
  file.write(sh)
!bash script.sh

In [ ]:
# Run on Onto Transfer
# No ensemble
#     --num_models 1 --backbone_dir_s1 out_wikigold_modified --do_strong_tuning --do_strong_tuning_eval --eval_on "test" | tee $OUT_DIR/train_log.txt
# Ensemble
#     --num_models 5 --backbone_dir out_onto_transfer_saved --do_train --do_eval --eval_on "test" | tee $OUT_DIR/train_log.txt

sh = """
CORPUS=onto_w_bio_w_6000
SEED=30
TEMP_DIR=tmp_${CORPUS}_$SEED
OUT_DIR=out_$CORPUS
mkdir -p $TEMP_DIR
mkdir -p $OUT_DIR

python -u src/train.py --data_dir new_data/onto_plus/$CORPUS --output_dir $OUT_DIR --temp_dir $TEMP_DIR \
    --pretrained_model roberta-base --tag_scheme 'io' --max_seq_length 150 \
    --train_batch_size 32 --gradient_accumulation_steps 2 --eval_batch_size 64 \
    --noise_train_lr 3e-5 --ensemble_train_lr 1e-5 --self_train_lr 5e-7 \
    --noise_train_epochs 3 --ensemble_train_epochs 2 --self_train_epochs 1 \
    --noise_train_update_interval 60 --self_train_update_interval 100 \
    --dropout 0.1 --warmup_proportion 0.1 --seed $SEED \
    --q 0.7 --tau 0.7 --num_models 1 --backbone_dir_s1 tmp_onto_strong_30 --do_strong_tuning --do_strong_tuning_eval --eval_on "test" | tee $OUT_DIR/train_log.txt
"""
with open('script.sh', 'w') as file:
  file.write(sh)

!bash script.sh

In [ ]:
# New Onto Transfer
# No ensemble
#     --num_models 1 --backbone_dir_s1 out_wikigold_modified --do_strong_tuning --do_strong_tuning_eval --eval_on "test" | tee $OUT_DIR/train_log.txt
# Ensemble
#     --num_models 5 --backbone_dir out_onto_transfer_saved --do_train --do_eval --eval_on "test" | tee $OUT_DIR/train_log.txt
sh = """
CORPUS=onto_w_bio_s_100
SEED=30
TEMP_DIR=tmp_${CORPUS}_$SEED
OUT_DIR=out_${CORPUS}
mkdir -p $TEMP_DIR
mkdir -p $OUT_DIR

python -u src/train.py --data_dir new_data/onto_plus/$CORPUS --output_dir $OUT_DIR --temp_dir $TEMP_DIR \
    --pretrained_model roberta-base --tag_scheme 'io' --max_seq_length 150 \
    --train_batch_size 32 --gradient_accumulation_steps 2 --eval_batch_size 64 \
    --noise_train_lr 3e-5 --ensemble_train_lr 1e-5 --self_train_lr 5e-7 \
    --noise_train_epochs 30 --ensemble_train_epochs 10 --self_train_epochs 5 \
    --noise_train_update_interval 60 --self_train_update_interval 100 \
    --dropout 0.1 --warmup_proportion 0.1 --seed $SEED \
    --q 0.0 --tau 0.0 --num_models 1 --backbone_dir_s1 out_onto_w --do_strong_tuning --do_strong_tuning_eval --eval_on "test" | tee $OUT_DIR/train_log.txt
"""
with open('script.sh', 'w') as file:
  file.write(sh)
!bash script.sh

In [ ]:
# Biocaster Transfer
# No ensemble
#     --num_models 1 --backbone_dir_s1 out_wikigold_modified --do_strong_tuning --do_strong_tuning_eval --eval_on "test" | tee $OUT_DIR/train_log.txt
# Ensemble
#     --num_models 5 --backbone_dir out_onto_transfer_saved --do_train --do_eval --eval_on "test" | tee $OUT_DIR/train_log.txt

sh = """
CORPUS=onto_w_bio_s_1000
SEED=30
TEMP_DIR=tmp_${CORPUS}_$SEED
OUT_DIR=out_$CORPUS
mkdir -p $TEMP_DIR
mkdir -p $OUT_DIR

python -u src/train.py --data_dir new_data/onto_plus/$CORPUS --output_dir $OUT_DIR --temp_dir $TEMP_DIR \
    --pretrained_model roberta-base --tag_scheme 'io' --max_seq_length 150 \
    --train_batch_size 32 --gradient_accumulation_steps 2 --eval_batch_size 64 \
    --noise_train_lr 3e-5 --ensemble_train_lr 1e-5 --self_train_lr 5e-7 \
    --noise_train_epochs 30 --ensemble_train_epochs 2 --self_train_epochs 1 \
    --noise_train_update_interval 60 --self_train_update_interval 100 \
    --dropout 0.1 --warmup_proportion 0.1 --seed $SEED \
    --q 0.0 --tau 0.0 --num_models 1 --backbone_dir_s1 out_onto_w --do_strong_tuning --do_strong_tuning_eval --eval_on "test" | tee $OUT_DIR/train_log.txt
"""
with open('script.sh', 'w') as file:
  file.write(sh)

!bash script.sh

In [ ]:
# Run on Medical Biocaster Transfer
# No ensemble
#     --num_models 1 --backbone_dir_s1 out_wikigold_modified --do_strong_tuning --do_strong_tuning_eval --eval_on "test" | tee $OUT_DIR/train_log.txt
# Ensemble
#     --num_models 5 --backbone_dir out_onto_transfer_saved --do_train --do_eval --eval_on "test" | tee $OUT_DIR/train_log.txt

sh = """
CORPUS=w_med
SEED=30
TEMP_DIR=tmp_${CORPUS}_$SEED
OUT_DIR=out_${CORPUS}
mkdir -p $TEMP_DIR
mkdir -p $OUT_DIR

python -u src/train.py --data_dir data/temp/$CORPUS --output_dir $OUT_DIR --temp_dir $TEMP_DIR \
    --pretrained_model roberta-base --tag_scheme 'io' --max_seq_length 150 \
    --train_batch_size 32 --gradient_accumulation_steps 2 --eval_batch_size 64 \
    --noise_train_lr 3e-5 --ensemble_train_lr 1e-5 --self_train_lr 5e-7 \
    --noise_train_epochs 3 --ensemble_train_epochs 2 --self_train_epochs 1 \
    --noise_train_update_interval 60 --self_train_update_interval 100 \
    --dropout 0.1 --warmup_proportion 0.1 --seed $SEED \
    --num_models 4 --backbone_dir out_w_med --do_train --do_eval --eval_on "test" | tee $OUT_DIR/train_log.txt
"""
with open('script.sh', 'w') as file:
  file.write(sh)


!bash script.sh

In [ ]:
# Run on Medical Biocaster Transfer
# No ensemble
#     --num_models 1 --backbone_dir_s1 out_wikigold_modified --do_strong_tuning --do_strong_tuning_eval --eval_on "test" | tee $OUT_DIR/train_log.txt
# Ensemble
#     --num_models 5 --backbone_dir out_onto_transfer_saved --do_train --do_eval --eval_on "test" | tee $OUT_DIR/train_log.txt

sh = """
CORPUS=w_med_w_biocaster_2775_s_100
SEED=30
TEMP_DIR=tmp_${CORPUS}_$SEED
OUT_DIR=out_${CORPUS}
mkdir -p $TEMP_DIR
mkdir -p $OUT_DIR

python -u src/train.py --data_dir data/temp/$CORPUS --output_dir $OUT_DIR --temp_dir $TEMP_DIR \
    --pretrained_model roberta-base --tag_scheme 'io' --max_seq_length 150 \
    --train_batch_size 32 --gradient_accumulation_steps 2 --eval_batch_size 64 \
    --noise_train_lr 3e-5 --ensemble_train_lr 1e-5 --self_train_lr 5e-7 \
    --noise_train_epochs 5 --ensemble_train_epochs 10 --self_train_epochs 5 \
    --noise_train_update_interval 60 --self_train_update_interval 100 \
    --dropout 0.1 --warmup_proportion 0.1 --seed $SEED \
    --num_models 1 --backbone_dir_s1 out_w_med_w_biocaster_2775 --do_strong_tuning --do_strong_tuning_eval --eval_on "test" | tee $OUT_DIR/train_log.txt
"""
with open('script.sh', 'w') as file:
  file.write(sh)


!bash script.sh